In [1]:
pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp39-cp39-macosx_10_9_universal2.whl size=1016126 sha256=94862d04a1a772695a532dd54920a0ab7ab81701c5f65820be50d159c4dda52e
  Stored in directory: /Users/huiyisang/Library/Caches/pip/wheels/42/41/d3/a56ae864ad22cc6583ec9312be43fbc611c87e53dc49aac953
Successfully built scikit-surprise

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [32]:
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

In [33]:
data = pd.merge(ratings_df, movies_df, on='movieId')

In [35]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(data[['userId','movieId','rating']], reader)

In [38]:
sim_options = {'name':'cosine',
               'user_based': False,
               'k':10} # content based filtering

In [39]:
algo = KNNBasic(sim_options=sim_options)

In [40]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9770  0.9711  0.9728  0.9792  0.9741  0.9748  0.0029  
Fit time          2.17    1.76    1.81    1.66    1.63    1.81    0.20    
Test time         2.72    2.57    2.58    2.53    2.75    2.63    0.09    


{'test_rmse': array([0.97695586, 0.97106302, 0.97280453, 0.97916213, 0.97414489]),
 'fit_time': (2.174401044845581,
  1.7640299797058105,
  1.8085310459136963,
  1.6581389904022217,
  1.6305081844329834),
 'test_time': (2.7179181575775146,
  2.567753314971924,
  2.576827049255371,
  2.5271310806274414,
  2.7454068660736084)}

In [9]:
algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [10]:
print(algo.sim)

[[1.         0.90773108 0.96657344 ... 1.         1.         1.        ]
 [0.90773108 1.         0.88581521 ... 1.         1.         1.        ]
 [0.96657344 0.88581521 1.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]


In [11]:
similarities = algo.sim

In [12]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [13]:
pred = algo.predict(uid=1, iid=10)
pred.est

3.9880309408033616

In [14]:
df[df['movieId']==10]

,userId,movieId,rating,timestamp,title,genres
468,7,10,4.0,1322062970,GoldenEye (1995),Action|Adventure|Thriller
690,9,10,3.0,842686329,GoldenEye (1995),Action|Adventure|Thriller
833,11,10,3.0,850816421,GoldenEye (1995),Action|Adventure|Thriller
2390,28,10,3.0,884100808,GoldenEye (1995),Action|Adventure|Thriller
2529,29,10,4.0,836583421,GoldenEye (1995),Action|Adventure|Thriller
...,...,...,...,...,...,...
94884,627,10,3.5,1237934561,GoldenEye (1995),Action|Adventure|Thriller
97622,650,10,5.0,965517712,GoldenEye (1995),Action|Adventure|Thriller
98091,653,10,4.0,829071627,GoldenEye (1995),Action|Adventure|Thriller
98262,656,10,4.0,838777501,GoldenEye (1995),Action|Adventure|Thriller


In [15]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [16]:
trainset, testset = train_test_split(data, test_size=0.25)

In [17]:
algo = KNNBasic(sim_options={'user_based':True})

In [18]:
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [19]:
predictions = algo.test(testset)

In [20]:
df[(df['userId']==232) & (df['movieId']==56367)]

,userId,movieId,rating,timestamp,title,genres
34153,232,56367,4.0,1277406098,Juno (2007),Comedy|Drama|Romance


In [21]:
predictions

[Prediction(uid=518, iid=1777, r_ui=2.0, est=3.314283192631024, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=668, iid=5459, r_ui=2.0, est=2.834142070388968, details={'actual_k': 37, 'was_impossible': False}),
 Prediction(uid=509, iid=3252, r_ui=3.0, est=3.765623021791895, details={'actual_k': 22, 'was_impossible': False}),
 Prediction(uid=354, iid=733, r_ui=4.0, est=3.38812666387841, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=413, iid=90405, r_ui=2.5, est=3.406468459370495, details={'actual_k': 11, 'was_impossible': False}),
 Prediction(uid=668, iid=7020, r_ui=3.0, est=3.716340565867206, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=232, iid=106918, r_ui=4.5, est=3.6557104888399885, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=268, iid=1639, r_ui=2.5, est=3.42688334031675, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=642, iid=107348, r_ui=1.0, est=3.0, details={'actual_k': 

In [22]:
df[df['userId']==1]['movieId'].unique()

array([   16,    24,    32,    47,    50,   110,   150,   161,   165,
         204,   223,   256,   260,   261,   277,   296,   318,   349,
         356,   377,   380,   457,   480,   527,   589,   590,   592,
         593,   597,   608,   648,   719,   724,   736,   780,   858,
         912,   968,  1061,  1089,  1136,  1196,  1198,  1210,  1213,
        1220,  1221,  1222,  1233,  1243,  1258,  1265,  1267,  1270,
        1287,  1580,  1617,  1721,  1923,  1961,  2021,  2028,  2105,
        2161,  2194,  2396,  2407,  2455,  2467,  2502,  2542,  2571,
        2628,  2716,  2728,  2762,  2858,  2947,  2959,  3256,  3421,
        3578,  4011,  4027,  4033,  4085,  4262,  4306,  4963,  4993,
        4995,  5349,  5378,  5418,  5445,  5952,  6365,  6711,  6807,
        7153,  8825,  8961, 32587, 33493, 33794, 33834, 45950, 48516,
       48780, 49272, 52973, 54286, 57949])

In [23]:
algo.predict(uid=1, iid=10)

Prediction(uid=1, iid=10, r_ui=None, est=3.534707054388631, details={'actual_k': 40, 'was_impossible': False})

In [24]:
df[df['movieId']==10]

,userId,movieId,rating,timestamp,title,genres
468,7,10,4.0,1322062970,GoldenEye (1995),Action|Adventure|Thriller
690,9,10,3.0,842686329,GoldenEye (1995),Action|Adventure|Thriller
833,11,10,3.0,850816421,GoldenEye (1995),Action|Adventure|Thriller
2390,28,10,3.0,884100808,GoldenEye (1995),Action|Adventure|Thriller
2529,29,10,4.0,836583421,GoldenEye (1995),Action|Adventure|Thriller
...,...,...,...,...,...,...
94884,627,10,3.5,1237934561,GoldenEye (1995),Action|Adventure|Thriller
97622,650,10,5.0,965517712,GoldenEye (1995),Action|Adventure|Thriller
98091,653,10,4.0,829071627,GoldenEye (1995),Action|Adventure|Thriller
98262,656,10,4.0,838777501,GoldenEye (1995),Action|Adventure|Thriller


In [25]:
# SVD

from surprise import SVD

In [26]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8664  0.8725  0.8638  0.8738  0.8704  0.8694  0.0038  
Fit time          0.84    0.77    0.86    0.89    0.84    0.84    0.04    
Test time         0.19    0.07    0.17    0.19    0.06    0.14    0.06    


{'test_rmse': array([0.86643903, 0.87253704, 0.86377456, 0.87382289, 0.87038897]),
 'fit_time': (0.8375539779663086,
  0.7722690105438232,
  0.8627560138702393,
  0.8889751434326172,
  0.8406670093536377),
 'test_time': (0.18682408332824707,
  0.0699300765991211,
  0.16794204711914062,
  0.1926729679107666,
  0.059625864028930664)}

In [27]:
trainset2 = data.build_full_trainset()
svd.fit(trainset)

In [28]:
svd.predict(1, 10)

Prediction(uid=1, iid=10, r_ui=None, est=3.5275054904938745, details={'was_impossible': False})

In [29]:
# predictions for everyone

df['predicted_rating'] = df.apply(lambda x: svd.predict(x['userId'], x['movieId']).est, axis=1)

In [30]:
df

,userId,movieId,rating,timestamp,title,genres,predicted_rating
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama,3.820280
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi,2.521215
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,4.057779
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.006199
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.402655
...,...,...,...,...,...,...,...
105334,668,142488,4.0,1451535844,Spotlight (2015),Thriller,2.740919
105335,668,142507,3.5,1451535889,Pawn Sacrifice (2015),Drama,2.928884
105336,668,143385,4.0,1446388585,Bridge of Spies (2015),Drama|Thriller,2.977016
105337,668,144976,2.5,1448656898,Bone Tomahawk (2015),Horror|Western,2.805546
